# Laboratorio 10 - Data Science
## Proyecto de Consultoría: Regresión Lineal con Python y Spark

- Nelson García Bravatti
- Christian Echeverría


In [ ]:
!pip -q install pyspark

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Lab Spark DF — Walmart").getOrCreate()
print("Spark version:", spark.version)